<img src = "https://drive.google.com/uc?export=view&id=1VV2e_u46fNm_ewns8QW2HGRZAPHh-e2t" alt = "Encabezado MLDS" width = "100%">  </img>

# **Diseño e implementación experimental**
---

Este notebook es una plantilla que le puede servir como guía para el tercer entregable del proyecto aplicado.


Proyecto aplicado por:


*   Cristhian David Mora Uribe cdmorau@unal.edu.co
*   Martin Camilo Rodriguez Murcia mrodriguezmu@unal.edu.co
*   Nestor Steven Negrete Pinilla narutones98@gmail.com

## **1. Particion del conjunto de datos**
---

Si el dataset no tiene por defecto definida una partición en conjuntos de entrenamiento y prueba, hágala usted, en las proporciones que considere oportunas.

In [ ]:
# ---**INGRESE SU CÓDIGO**---

from google.colab import drive
drive.mount('/content/drive')
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/DeepLearning/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Descargar el dataset de "Twitter Financial News" desde Kaggle y extraerlo
!kaggle datasets download sudalairajkumar/cryptocurrencypricehistory

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/sudalairajkumar/cryptocurrencypricehistory
License(s): CC0-1.0
  0% 0.00/1.70M [00:00<?, ?B/s]
100% 1.70M/1.70M [00:00<00:00, 177MB/s]


In [ ]:
import os
import zipfile
import pandas as pd

# Extraer el archivo zip descargado
with zipfile.ZipFile("/content/cryptocurrencypricehistory.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/crypto_data")

# Ruta donde están los archivos extraídos
folder_path = "/content/crypto_data/"

# Lista de los archivos CSV que quieres leer
file_names = [
    "coin_Aave.csv", "coin_BinanceCoin.csv", "coin_Bitcoin.csv", "coin_Cardano.csv",
    "coin_ChainLink.csv", "coin_Cosmos.csv", "coin_CryptocomCoin.csv", "coin_Dogecoin.csv",
    "coin_EOS.csv", "coin_Ethereum.csv", "coin_Iota.csv", "coin_Litecoin.csv",
    "coin_Monero.csv", "coin_NEM.csv", "coin_Polkadot.csv", "coin_Solana.csv",
    "coin_Stellar.csv", "coin_Tether.csv", "coin_Tron.csv", "coin_USDCoin.csv",
    "coin_Uniswap.csv", "coin_WrappedBitcoin.csv", "coin_XRP.csv"
]

# Diccionario para almacenar los DataFrames
dataframes = {}

# Leer cada archivo CSV y almacenarlo en el diccionario
for file in file_names:
    file_path = os.path.join(folder_path, file)
    coin_name = file.replace("coin_", "").replace(".csv", "")  # Extraer el nombre de la criptomoneda
    dataframes[coin_name] = pd.read_csv(file_path)



In [ ]:
from sklearn.model_selection import train_test_split
bitcoin_data = dataframes['Bitcoin']

bitcoin_data['Date'] = pd.to_datetime(bitcoin_data['Date'])
bitcoin_data = bitcoin_data.sort_values('Date')

# Definir las características y la variable objetivo
X = bitcoin_data[['Open', 'High', 'Low', 'Volume']]  # Características
y = bitcoin_data['Close']  # Variable objetivo

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=False)

print("Conjunto de Entrenamiento:")
print(X_train.head())
print("\nConjunto de Prueba:")
print(X_test.head())

Conjunto de Entrenamiento:
         Open        High         Low  Volume
0  134.444000  147.488007  134.000000     0.0
1  144.000000  146.929993  134.050003     0.0
2  139.000000  139.889999  107.720001     0.0
3  116.379997  125.599998   92.281898     0.0
4  106.250000  108.127998   79.099998     0.0

Conjunto de Prueba:
              Open          High           Low        Volume
2243   9273.060783   9594.420276   9232.484263  1.784682e+10
2244   9525.074608  10144.556717   9525.074608  2.062401e+10
2245  10175.923956  11157.345516  10107.035204  2.999520e+10
2246  10696.690929  11246.144183  10556.095932  2.099833e+10
2247  10853.743838  11065.896018  10610.428082  1.927165e+10


## **2. Selección y diseño de modelos**
---

Seleccione el/los modelo/s a explorar para aplicar sobre el conjunto de datos. Recuerde, la selección del modelo está influenciada por diferentes factores. Si el problema es de análisis de imagen, muy seguramente hay que explorar diversas redes neuronales convolucionales. Si el problema está relacionado con NLP, muy seguramente hay que explorar modelos basados en Tranformers.

Además, debe definir cuál es su problema:

- **Regresión**: se busca estimar un valor continúo a partir de los datos.
- **Clasificación**: permite estimar un valor categórico a partir de los datos.
- **Agrupamiento**: permite encontrar grupos de datos similares.
- **Otros modelos**: recuerde que dispone de otros tipos de tareas supervisadas y no supervisadas.

En cualquier caso los profundos disponibles en _TensorFlow_ constituyen una base sobre la que usted debe definir un clasificador/regresor/_encoder_/_decoder_ final, compuesto de una o más capas densas, con opción de incluir _dropout_ o capas de normalización.

Justifique la escogencia y diseño de los modelos a explorar:

# Justificación

Teniendo en cuenta nuestro objetivo de predecir el precio de cierre de criptomonedas (Bitcoin, para ser más especificos), el modelo que vamos a trabajar es **CNN-LSTM**. Es decir, la idea es combinar modelos de Redes Neuronales Convuncionales con Redes Neuronales Recurrentes.

La escogencia de este modelo se da por 3 razones principales:

- Debido a la alta volatilidad de las series de tiempo de criptomonedas, es necesario capturar patrones a corto plazo. Para lograr este objetivo, son de suma utilidad los modelos CNN. Por ello, en nuestro modelo se incluiran inicialmente capas convulcionales 1D, por medio del modelo **Conv1D** de TensorFlow.

- Adicional a los patrones de corto plazo, es de vital importancia identificar y capturar las tendencias a largo plazo de las series de tiempo, para lo cual recurrimos al modelo **LSTM**. De esta forma, utilizamos las carateristicas extraidas de las capas Conv1D y las pasamos a una capa LSTM.

- Si bien podriamos hacer uso de Transformers para capturar más adecuadamente el comportamiento de nuestra serie de tiempo, esto requeriria una gran cantidad de recursos, por lo que, en pro de la eficiencia, consideramos más apropiado recurrir a los modelos mencionados anteriormente.



## **3. Implementación de los modelos**
---

Implemente los modelos descritos anteriormente usando herramientas de _TensorFlow_. Recuerde que puede aplicar técnicas de aumentación de datos, si es necesario.

In [ ]:
# ---**INGRESE SU CÓDIGO**---

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Suponemos que tenemos el DataFrame 'dataframes['Bitcoin']' cargado y que contiene el precio de cierre en una columna 'Close'

# 1. Preprocesamiento de los datos
# Escalar los datos para que estén en el rango [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(dataframes['Bitcoin'][['Close']].values)

# Definir el tamaño de la ventana de tiempo
window_size = 60  # Número de días para mirar hacia atrás en cada predicción

# Preparar los datos para el modelo CNN-LSTM
X = []
y = []

for i in range(window_size, len(data)):
    X.append(data[i - window_size:i, 0])
    y.append(data[i, 0])

X, y = np.array(X), np.array(y)

# Cambiar la forma de X para que sea compatible con Conv1D y LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# 2. Definir el modelo CNN-LSTM
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dense(units=1)  # Salida de una única neurona para la predicción del precio de cierre
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# 3. Entrenar el modelo
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# 4. Evaluar el modelo y realizar predicciones
# Para predicciones, desnormalizar los valores de salida
predicted_prices = model.predict(X)
predicted_prices = scaler.inverse_transform(predicted_prices)  # Convertir de nuevo a los valores originales

# Ejemplo: mostrar la última predicción
print("Última predicción del precio:", predicted_prices[-1])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 0.0012 - val_loss: 0.0040
Epoch 2/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 1.0339e-04 - val_loss: 0.0069
Epoch 3/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.5598e-05 - val_loss: 0.0082
Epoch 4/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.0300e-04 - val_loss: 0.0054
Epoch 5/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 8.2132e-05 - val_loss: 0.0071
Epoch 6/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.2750e-04 - val_loss: 0.0046
Epoch 7/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7.5740e-05 - val_loss: 0.0080
Epoch 8/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.0239e-05 - val_loss: 0.0071
Epoch 9/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.4377e-05 - val_loss: 0.0076
Epoch 10/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.8796e-05 - val_loss: 0.0086
Epoch 11/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.7581e-05 - val_loss: 0.0059
Epoch 12/50
74/74 ━━━━━━━━━━

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*